In [25]:
!docker run -d --name nginx --network=none -p 81:81 nginx


ca52a1b463fbae692ec48efa24c14ef1b26ffcf9b400a0974b061879c27db80e


In [26]:
!docker exec -it nginx2 /bin/sh -c "curl google.com"

Error response from daemon: No such container: nginx2


In [27]:
!docker network


Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


In [28]:

!docker network ls 


NETWORK ID     NAME          DRIVER    SCOPE
266ec01e86ea   bridge        bridge    local
54411b948e85   host          host      local
4778c89a25b2   none          null      local
eddfab0229a8   yourNetwork   bridge    local


In [29]:
!docker network inspect none

[
    {
        "Name": "none",
        "Id": "4778c89a25b200f8e9d3abe5c86481968e7f15d0d828536d1bf574084ad752a3",
        "Created": "2025-03-15T03:21:00.220417292Z",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "ca52a1b463fbae692ec48efa24c14ef1b26ffcf9b400a0974b061879c27db80e": {
                "Name": "nginx",
                "EndpointID": "62f3e61f9015f1afe7a3c61c62788508841866608cda72bbb7b66ffe57df3c91",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]


In [30]:
!pip install docker


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])

Docker version: 27.5.1


In [32]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

nginx - running


In [33]:

#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")



nginx in network 'none' has IP: 


In [34]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

7b69419b7d63b6276f36999f784e056ec2a3fc8c259aa5c53a208c52ce47d3f7


In [35]:
!docker run -d --name adminer -p 8080:8080 adminer

4b4c36c079d7189865d77dd00895956807c2e47cc466ce7010f1b1e87dc8c12f


In [36]:
!docker inspect mysql | grep -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("mysql")

------------------------------
mysql in network 'bridge' has IP: 172.17.0.2


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [37]:
!docker inspect adminer| grep  -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("adminer")

------------------------------
adminer in network 'bridge' has IP: 172.17.0.3


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [38]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()
        container.reload()
        if container.status == 'exited':
            print(f"Removing {container.name}")
            container.remove()

list_running_containers()

clean_containers()

adminer - running
mysql - running
nginx - running
Removing adminer
Removing mysql
Removing nginx


In [39]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

Error response from daemon: network with name yourNetwork already exists


In [40]:


!docker network inspect yourNetwork



[
    {
        "Name": "yourNetwork",
        "Id": "eddfab0229a82a9ae919281b1bc6de0e3ecee705a11968b8744a34f4575330a8",
        "Created": "2025-04-19T04:14:53.827988627Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


In [41]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8

8f958985d38729507b3a0024338b7907e0d8568c85f58aeca81a564aa38fef1a


In [42]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

4fbf1e7cd0fba60d44e94cef3774895825433b504e5bf4cfe5bf02d0cc4caef7


In [43]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [44]:
!docker inspect mysql |grep -C2 " \"Networks\""
!docker inspect mysql |grep -A9 " \"Networks\""| grep " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

'grep' is not recognized as an internal or external command,
operable program or batch file.


------------------------------
mysql in network 'yourNetwork' has IP: 172.18.0.2


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [45]:
!docker inspect adminer| grep -A2 " \"Networks\""
!docker inspect adminer| grep -A9 " \"Networks\""| grep " \"IPAddress\""
!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

'grep' is not recognized as an internal or external command,
operable program or batch file.
'grep' is not recognized as an internal or external command,
operable program or batch file.


Access denied. Option -c requires administrative privileges.------------------------------

adminer in network 'yourNetwork' has IP: 172.18.0.3
